In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler,StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt
orders=pd.read_csv('/content/drive/MyDrive/ml_project/orders.csv')
orders.head()
#print(orders.info(memory_usage='deep')) i can use also orders.info() in general

In [ ]:
orders['order_id']=orders['order_id'].astype('int32')
orders['user_id']=orders['user_id'].astype('int32')
orders['order_number']=orders['order_number'].astype('int8')
orders['eval_set']=orders['eval_set'].astype('category')

In [ ]:
orders['days_since_prior_order']=orders['days_since_prior_order'].fillna(0)
#orders.head()
orders=orders.drop_duplicates()

to handle the missing value in the days since prior order column we did the imputation, because the missing value (NaN) happens when there isn't a previous order that exists to calculate the time gap, then we dropped the duplicated columns to make sure the data is clean.

In [ ]:
order_prior = pd.read_csv('/content/drive/MyDrive/ml_project/order_products__prior.csv')

In [ ]:
order_prior.head()

In [ ]:
order_prior['order_id']=order_prior['order_id'].astype('int32')
order_prior['product_id']=order_prior['product_id'].astype('int32')
order_prior['reordered']=order_prior['reordered'].astype('int8')

since RAM has a certain limit we have to downcast due to large dataset size

In [ ]:
order_prior.info()

In [ ]:
order_prior=order_prior[['order_id','product_id','reordered']]

In [ ]:
products_per_order =(order_prior.groupby('order_id').size().reset_index(name='num_products'))

In [ ]:
orders =orders.merge(products_per_order,on='order_id',how='left')

In [ ]:
orders.head()

In [ ]:
orders[['order_dow','order_hour_of_day','num_products']].describe()

In [ ]:
plt.figure()
sns.histplot(orders['num_products'], bins=30)
plt.title('Distribution of Number of Products per Order')
plt.xlabel('Number of Products')
plt.ylabel('Count')
plt.show()

The majority of orders contain a small number of products,with a right skewed distribution.This indicates that most customers place small orders, while large orders are relatively rare.



In [ ]:
reorder_rate_per_order =(order_prior.groupby('order_id')['reordered'].mean().reset_index(name='reorder_rate'))

In [ ]:
orders = orders.merge(reorder_rate_per_order, on='order_id', how='left')

In [ ]:
orders.head()

In [ ]:
orders['reorder_rate'].describe()

In [ ]:
plt.figure()
sns.histplot(orders['reorder_rate'], bins=30)
plt.title('Distribution of Reorder Rate per Order')
plt.xlabel('Reorder Rate')
plt.ylabel('Count')
plt.show()


Most orders have a low reorder rate, indicating that many orders contain new products.However, a subset of orders shows high reorder rates, suggesting consistent purchasing behavior.


In [ ]:
full_prior_orders=pd.merge(order_prior,orders,on='order_id',how='left')

In [ ]:
full_prior_orders.head()

In [ ]:
full_prior_orders.to_csv('/content/drive/MyDrive/ml_project/full_prior_orders.csv',index=False)